In [87]:
import yfinance as yf
import pandas as pd
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import time

In [88]:
stocks = pd.read_excel("Planilha de Ouro Stocks - Fevereiro 2025.xlsx")
reits = pd.read_excel("Planilha de Ouro REITs - Fevereiro 2025.xlsx")

In [89]:
import pandas as pd

def filter_stocks(stocks: pd.DataFrame) -> list:
    """
    Filtra o DataFrame de stocks com base nos critérios especificados e retorna uma lista de tickers que atendem a todas as condições.
    Imprime o número de stocks que atendem aos critérios.

    Critérios:
    - Anos sem prejuízo >= 14
    - CAGR do LPA em 5 anos >= 0.10
    - Dívida Líquida/EBITDA <= 5.0
    - ROE >= 0.10
    - ROIC >= 0.04
    - Margem Líquida >= 0.07
    - Margem EBIT >= 0.07
    - Anos desde o IPO >= 14

    Args:
        stocks (pd.DataFrame): DataFrame contendo dados das ações.

    Returns:
        list: Lista de tickers que atendem aos critérios.
    """
    # Aplicar os filtros
    filtered_stocks = stocks[
        (stocks['Anos sem prejuízo'] >= 14) &
        (stocks['CAGR do LPA em 5 anos'] >= 0.10) &
        (stocks['Dívida Líquida/EBITDA'] <= 5.0) &
        (stocks['ROE'] >= 0.10) &
        (stocks['ROIC'] >= 0.04) &
        (stocks['Margem Líquida'] >= 0.07) &
        (stocks['Margem EBIT'] >= 0.07) &
        (stocks['Anos desde o IPO'] >= 14)
    ]
    
    # Contar o número de stocks filtrados
    count = len(filtered_stocks)
    
    # Imprimir o número de stocks filtrados
    print(f"Number of stocks that meet the criteria: {count}")
    
    # Retornar a lista de tickers
    return filtered_stocks['Ticker'].tolist()

def filter_reits(reits: pd.DataFrame) -> list:
    """
    Filtra o DataFrame de REITs com base nos critérios especificados e retorna uma lista de tickers que atendem a todas as condições.
    Imprime o número de REITs que atendem aos critérios.

    Critérios:
    - Anos sem prejuízo >= 10
    - CAGR do LPA em 5 anos >= 0.04
    - Dívida Líquida/EBITDA <= 6.5
    - ROE >= 0.05
    - Margem Líquida >= 0.10
    - Margem EBIT >= 0.10
    - Anos desde o IPO >= 10

    Args:
        reits (pd.DataFrame): DataFrame contendo dados dos REITs.

    Returns:
        list: Lista de tickers que atendem aos critérios.
    """
    # Aplicar os filtros
    filtered_reits = reits[
        (reits['Anos sem prejuízo'] >= 10) &
        (reits['CAGR do LPA em 5 anos'] >= 0.04) &
        (reits['Dívida Líquida/EBITDA'] <= 6.5) &
        (reits['ROE'] >= 0.05) &
        (reits['Margem Líquida'] >= 0.10) &
        (reits['Margem EBIT'] >= 0.10) &
        (reits['Anos desde o IPO'] >= 10)
    ]
    
    # Contar o número de REITs filtrados
    count = len(filtered_reits)
    
    # Imprimir o número de REITs filtrados
    print(f"Number of REITs that meet the criteria: {count}")
    
    # Retornar a lista de tickers
    return filtered_reits['Ticker'].tolist()

In [90]:
filtered_stocks_list = filter_stocks(stocks)
filtered_reits_list = filter_reits(reits)
print("Stocks filtradas:", filtered_stocks_list)
print("REITs filtradas:", filtered_reits_list)

Number of stocks that meet the criteria: 121
Number of REITs that meet the criteria: 17
Stocks filtradas: ['WSO', 'WSM', 'WM', 'WAB', 'VRSK', 'VMI', 'V', 'USLM', 'URI', 'ULH', 'UFPT', 'TXRH', 'TSCO', 'TRMB', 'TOL', 'TMUS', 'TJX', 'TGNA', 'SYK', 'SSNC', 'SHW', 'SCI', 'SCCO', 'RSG', 'RPM', 'ROL', 'RMD', 'RJF', 'PH', 'NVDA', 'NFLX', 'MPWR', 'MORN', 'MMC', 'MLI', 'MCRI', 'MATX', 'MA', 'LPLA', 'LMAT', 'LII', 'KLAC', 'KAI', 'IT', 'ISRG', 'INGR', 'IDCC', 'IBKR', 'HUBB', 'GWW', 'GOOGL', 'FTNT', 'FI', 'EXLS', 'ETN', 'DVA', 'DOV', 'DORM', 'DKS', 'DE', 'CSL', 'CPRT', 'CMG', 'CME', 'CBZ', 'BRO', 'BMI', 'AYI', 'APH', 'APD', 'AMG', 'ACIW', 'AAPL', 'AAON', 'TMO', 'SAIA', 'PG', 'PCAR', 'NVR', 'NOC', 'MLM', 'GPK', 'ETD', 'DCI', 'CBOE', 'AMAT', 'WST', 'WSO.B', 'ULTA', 'SSD', 'SNPS', 'RS', 'POOL', 'PLAB', 'OLED', 'ODFL', 'NUE', 'NEU', 'MSFT', 'MGRC', 'MANH', 'LRCX', 'LEN.B', 'LEN', 'LECO', 'IPAR', 'INTU', 'IDXX', 'HOG', 'FELE', 'EW', 'DHI', 'CWT', 'BALL', 'BAH', 'AVY', 'ASR', 'AOS', 'ALG', 'ADBE', 'ACU']

In [91]:
def fetch_fundamental_data(tickers):
    fundamental_data = []
    seen_tickers = set()  # Track processed tickers
    for ticker in tickers:
        if ticker in seen_tickers:
            continue  # Skip duplicates
        try:
            ticker_obj = yf.Ticker(ticker)
            info = ticker_obj.info
            data = {
                'Ticker': ticker,
                'Company Name': info.get('longName', 'N/A'),
                'Sector': info.get('sector', 'N/A'),
                'Market Cap (B)': info.get('marketCap', 'N/A'),
                'P/E Ratio': info.get('trailingPE', 'N/A'),
                'Dividend Yield (%)': info.get('dividendYield', 'N/A'),
            }
            fundamental_data.append(data)
            seen_tickers.add(ticker)
        except Exception as e:
            print(f"Error fetching fundamental data for {ticker}: {e}")
    return pd.DataFrame(fundamental_data)

In [96]:
def fetch_historical_data(tickers, start_date, end_date):
    """Fetch historical data in batches to avoid API overload."""
    batch_size = 50
    all_data = []
    unique_tickers = list(dict.fromkeys(tickers))  # Preserve order, remove duplicates
    
    for i in range(0, len(unique_tickers), batch_size):
        batch = unique_tickers[i:i + batch_size]
        print(f"Fetching batch: {batch}")
        for attempt in range(3):
            try:
                data = yf.download(batch, start=start_date, end=end_date, 
                                 auto_adjust=False, progress=False)
                if isinstance(data.columns, pd.MultiIndex):
                    if 'Adj Close' not in data.columns.levels[0]:
                        raise KeyError("'Adj Close' not found in MultiIndex levels!")
                    data = data['Adj Close']
                elif 'Adj Close' not in data.columns:
                    raise KeyError("'Adj Close' not found in flat columns!")
                all_data.append(data)
                break
            except Exception as e:
                print(f"Attempt {attempt + 1} for batch {batch} failed: {e}. Retrying in 5 seconds...")
                time.sleep(5)
        else:
            print(f"Failed to fetch data for batch {batch} after 3 attempts.")
    if not all_data:
        raise ValueError("No historical data retrieved!")
    combined_data = pd.concat(all_data, axis=1)
    # Drop duplicate columns if any (e.g., from ticker mapping overlap)
    combined_data = combined_data.loc[:, ~combined_data.columns.duplicated()]
    return combined_data

def verify_performance_and_fundamentals(filtered_stocks, filtered_reits):
    ticker_mapping = {'LEN.B': 'LEN', 'WSO.B': 'WSO'}
    filtered_stocks = list(dict.fromkeys([ticker_mapping.get(t, t) for t in filtered_stocks]))
    filtered_reits = list(dict.fromkeys([ticker_mapping.get(t, t) for t in filtered_reits]))
    
    all_tickers = list(dict.fromkeys(filtered_stocks + filtered_reits))
    tickers_with_sp = list(dict.fromkeys(all_tickers + ['^GSPC']))
    
    end_date = date.today()
    start_date = end_date - relativedelta(years=20)
    print(f"Fetching data from {start_date} to {end_date}")
    
    data = fetch_historical_data(tickers_with_sp, start_date, end_date)
    
    print(f"Columns in historical data: {data.columns.tolist()}")
    if '^GSPC' not in data.columns:
        raise ValueError("S&P 500 (^GSPC) data not included in historical data.")
    
    time_frames = [1, 3, 5, 10, 15, 20]
    return_df = pd.DataFrame(index=data.columns)
    end_date = data.index[-1]
    print(f"Last available date: {end_date}")
    for N in time_frames:
        start_date_N = end_date - relativedelta(years=N)
        start_prices = data.asof(start_date_N)
        end_prices = data.iloc[-1]
        return_Ny = (end_prices / start_prices) - 1
        return_df[f'{N}Y Return'] = return_Ny
    
    fundamental_df = fetch_fundamental_data(all_tickers)
    fundamental_df['Type'] = fundamental_df['Ticker'].apply(
        lambda x: 'Stock' if x in filtered_stocks else 'REIT'
    )
    
    fundamental_df['Market Cap (B)'] = fundamental_df['Market Cap (B)'].apply(
        lambda x: x / 1e9 if isinstance(x, (int, float)) and not pd.isna(x) else x
    )
    
    return_df = return_df.reset_index().rename(columns={'index': 'Ticker'})
    
    final_df = pd.merge(
        fundamental_df,
        return_df[return_df['Ticker'].isin(all_tickers)],
        on='Ticker'
    ).drop_duplicates(subset=['Ticker'])
    
    sp_data = return_df[return_df['Ticker'] == '^GSPC']
    if sp_data.empty:
        print("Warning: S&P 500 data (^GSPC) not found in historical data.")
        sp_returns = pd.Series(index=[f"{n}Y Return" for n in time_frames], data=[None] * len(time_frames))
    else:
        sp_returns = sp_data.iloc[0, 1:]
    
    # Format and display S&P 500 returns as percentages
    print("S&P 500 Returns:")
    for col, value in sp_returns.items():
        if pd.isna(value):
            print(f"{col:<12}: N/A")
        else:
            print(f"{col:<12}: {value * 100:.2f}%")
    
    # Format final DataFrame
    for col in [f'{n}Y Return' for n in time_frames]:
        final_df[col] = final_df[col].apply(lambda x: f"{x * 100:.2f}%" if not pd.isna(x) else 'N/A')
    final_df['Market Cap (B)'] = final_df['Market Cap (B)'].apply(
        lambda x: f"{x:.2f}" if isinstance(x, (int, float)) and not pd.isna(x) else 'N/A'
    )
    
    print("\nFundamental and Performance Data:")
    print(final_df.to_string(index=False))
    
    return final_df, sp_returns, data

In [ ]:
# Call the function
final_df, sp_returns, historical_data = verify_performance_and_fundamentals(
    filtered_stocks_list, filtered_reits_list
)



Fetching data from 2005-03-01 to 2025-03-01
Fetching batch: ['WSO', 'WSM', 'WM', 'WAB', 'VRSK', 'VMI', 'V', 'USLM', 'URI', 'ULH', 'UFPT', 'TXRH', 'TSCO', 'TRMB', 'TOL', 'TMUS', 'TJX', 'TGNA', 'SYK', 'SSNC', 'SHW', 'SCI', 'SCCO', 'RSG', 'RPM', 'ROL', 'RMD', 'RJF', 'PH', 'NVDA', 'NFLX', 'MPWR', 'MORN', 'MMC', 'MLI', 'MCRI', 'MATX', 'MA', 'LPLA', 'LMAT', 'LII', 'KLAC', 'KAI', 'IT', 'ISRG', 'INGR', 'IDCC', 'IBKR', 'HUBB', 'GWW']
Fetching batch: ['GOOGL', 'FTNT', 'FI', 'EXLS', 'ETN', 'DVA', 'DOV', 'DORM', 'DKS', 'DE', 'CSL', 'CPRT', 'CMG', 'CME', 'CBZ', 'BRO', 'BMI', 'AYI', 'APH', 'APD', 'AMG', 'ACIW', 'AAPL', 'AAON', 'TMO', 'SAIA', 'PG', 'PCAR', 'NVR', 'NOC', 'MLM', 'GPK', 'ETD', 'DCI', 'CBOE', 'AMAT', 'WST', 'ULTA', 'SSD', 'SNPS', 'RS', 'POOL', 'PLAB', 'OLED', 'ODFL', 'NUE', 'NEU', 'MSFT', 'MGRC', 'MANH']
Fetching batch: ['LRCX', 'LEN', 'LECO', 'IPAR', 'INTU', 'IDXX', 'HOG', 'FELE', 'EW', 'DHI', 'CWT', 'BALL', 'BAH', 'AVY', 'ASR', 'AOS', 'ALG', 'ADBE', 'ACU', 'PLD', 'EQIX', 'AVB', 'PSA', 

In [94]:
final_df

,Ticker,Company Name,Sector,Market Cap (B),P/E Ratio,Dividend Yield (%),Type,1Y Return,3Y Return,5Y Return,10Y Return,15Y Return,20Y Return
0,WSO,"Watsco, Inc.",Industrials,20.35,40.803400,2.14,Stock,30.32%,100.85%,272.22%,483.24%,N/A,N/A
1,WSM,"Williams-Sonoma, Inc.",Consumer Cyclical,23.95,23.000000,1.17,Stock,69.69%,186.17%,591.53%,516.72%,N/A,N/A
2,WM,"Waste Management, Inc.",Industrials,93.60,34.131966,1.42,Stock,13.92%,69.10%,128.22%,422.29%,N/A,N/A
3,WAB,Westinghouse Air Brake Technologies Corporation,Industrials,31.67,30.688742,0.54,Stock,33.52%,103.23%,178.15%,106.78%,N/A,N/A
4,VRSK,"Verisk Analytics, Inc.",Industrials,41.65,44.648120,0.61,Stock,23.42%,70.66%,97.55%,329.69%,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,FR,"First Industrial Realty Trust, Inc.",Real Estate,7.76,26.304148,3.12,REIT,11.16%,7.15%,67.94%,248.16%,N/A,N/A
132,RYN,Rayonier Inc.,Real Estate,4.19,11.083681,4.11,REIT,-13.27%,-20.95%,27.13%,47.70%,N/A,N/A
133,ELS,"Equity LifeStyle Properties, Inc.",Real Estate,13.73,34.989796,3.0,REIT,6.98%,-0.45%,13.47%,223.27%,N/A,N/A
134,LAMR,Lamar Advertising Company,Real Estate,12.72,35.292614,4.99,REIT,19.82%,32.32%,85.73%,240.86%,N/A,N/A


In [95]:
sp_returns

1Y Return     0.174513
3Y Return     0.361358
5Y Return     1.015591
10Y Return    1.829413
15Y Return         NaN
20Y Return         NaN
Name: 136, dtype: object